In [1]:
#brew install ffmpeg

import cv2
import pandas as pd
import numpy as np
import os

class Config:
    """Configuration for the eye tracking visualization."""
    
    def __init__(self):
        # Video paths
        self.video_path = "assets/Test Your Awareness.avi"
        self.data_path = "assets/DataPOR.csv"
        self.output_path = "assets/EyeTracking.avi"
        
        self._verify_files()
        
        # Resolution mapping
        self.source_resolution = (1600, 1050)
        
        # Point visualization settings
        self.point_config = {
            'color': (0, 0, 255),  # BGR format - Red
            'radius': 15,
            'thickness': -1,  # Filled circle
            'alpha': 0.6,     # Transparence pour le point
            'blur_size': (35, 35)  # Taille du flou gaussien
        }
        
        # Smoothing parameter (0 = no smoothing, 1 = maximum smoothing)
        self.smoothing_factor = 0.8

    def _verify_files(self):
        """Verify input files existence."""
        if not os.path.exists(self.video_path):
            raise FileNotFoundError(f"Missing video file: {self.video_path}")
        if not os.path.exists(self.data_path):
            raise FileNotFoundError(f"Missing data file: {self.data_path}")

class ExponentialSmoother:
    """Simple exponential smoothing filter for POR coordinates."""
    
    def __init__(self, alpha):
        self.alpha = alpha  # Smoothing factor (1-alpha actually applied to old value)
        self.last_value = None
    
    def update(self, measurement):
        """Update the filter with new measurement."""
        if self.last_value is None:
            self.last_value = np.array(measurement, dtype=float)
            return measurement
        
        # Apply exponential smoothing
        current_value = np.array(measurement, dtype=float)
        smooth_value = self.alpha * current_value + (1 - self.alpha) * self.last_value
        self.last_value = smooth_value
        
        return tuple(map(int, smooth_value))

class EyeTrackingDataProcessor:
    """Handles loading and processing of eye tracking data."""
    
    def __init__(self, config):
        self.config = config
        self.data = None
        self.frame_count = 0
        self.current_row = 0
        self.increment = 0
        # Initialize smoother with (1 - smoothing_factor) as alpha
        # because we want smoothing_factor to represent how much we keep from previous value
        self.smoother = ExponentialSmoother(1 - config.smoothing_factor)
        
    def load_data(self):
        """Load and prepare eye tracking data from CSV."""
        self.data = pd.read_csv(
            self.config.data_path,
            delimiter='\t',
            encoding='utf-8',
            low_memory=False
        )
        self.eye_data = self.data.loc[1:, ["L POR X [px]", "L POR Y [px]"]]
        
    def initialize_processing(self, frame_count):
        """Initialize processing parameters."""
        self.frame_count = frame_count
        self.increment = len(self.eye_data) / frame_count
        
    def get_coordinates(self, target_resolution):
        """Get smoothed coordinates, scaled to target resolution."""
        if int(self.current_row) >= len(self.eye_data) - 1:
            return None
            
        current = self._scale_coordinates(
            self.eye_data.iloc[int(self.current_row)],
            target_resolution
        )
        
        # Apply smoothing
        smoothed_coords = self.smoother.update(current)
        self.current_row += self.increment
        
        return smoothed_coords
        
    def _scale_coordinates(self, point_data, target_resolution):
        """Scale coordinates from source to target resolution."""
        x = float(point_data["L POR X [px]"]) / self.config.source_resolution[0] * target_resolution[0]
        y = float(point_data["L POR Y [px]"]) / self.config.source_resolution[1] * target_resolution[1]
        return (int(x), int(y))

class PointVisualizer:
    """Handles the visualization of eye tracking points on video frames."""
    
    def __init__(self, config):
        self.config = config
    
    def draw_point(self, frame, point):
        """Draw a smooth, semi-transparent point with blur effect."""
        if not self._are_coordinates_valid(point, frame.shape):
            return frame
            
        # Create overlay for the point
        overlay = frame.copy()
        
        # Draw the main point
        cv2.circle(
            overlay,
            point,
            self.config.point_config['radius'],
            self.config.point_config['color'],
            self.config.point_config['thickness']
        )
        
        # Apply gaussian blur to create a smooth effect
        overlay = cv2.GaussianBlur(
            overlay,
            self.config.point_config['blur_size'],
            0
        )
        
        # Blend the overlay with the original frame
        return cv2.addWeighted(
            frame,
            1 - self.config.point_config['alpha'],
            overlay,
            self.config.point_config['alpha'],
            0
        )
    
    def _are_coordinates_valid(self, point, frame_shape):
        """Check if coordinates are within frame boundaries."""
        return (point and 0 <= point[0] < frame_shape[1] and 
                0 <= point[1] < frame_shape[0])

class VideoProcessor:
    """Handles video input/output operations."""
    
    def __init__(self, config):
        self.config = config
        self.cap = None
        self.out = None
        
    def open_video(self):
        """Open input video and prepare output video writer."""
        self.cap = cv2.VideoCapture(self.config.video_path)
        
        # Get video properties
        frame_size = (
            int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        )
        fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter(
            self.config.output_path,
            fourcc,
            fps,
            frame_size
        )
        
        return frame_count, frame_size
        
    def read_frame(self):
        """Read a frame from input video."""
        return self.cap.read()
        
    def write_frame(self, frame):
        """Write a frame to output video."""
        self.out.write(frame)
        
    def release(self):
        """Release video resources."""
        if self.cap:
            self.cap.release()
        if self.out:
            self.out.release()
        cv2.destroyAllWindows()

def main():
    """Main application entry point."""
    config = Config()
    video_processor = VideoProcessor(config)
    data_processor = EyeTrackingDataProcessor(config)
    point_visualizer = PointVisualizer(config)
    
    try:
        # Setup
        frame_count, frame_size = video_processor.open_video()
        data_processor.load_data()
        data_processor.initialize_processing(frame_count)
        
        # Main processing loop
        while True:
            ret, frame = video_processor.read_frame()
            if not ret:
                break
            
            # Get smoothed coordinates
            coords = data_processor.get_coordinates(frame_size)
            if coords is not None:
                frame = point_visualizer.draw_point(frame, coords)
            
            video_processor.write_frame(frame)
            
    finally:
        video_processor.release()
        print(f"Eye tracking video saved to: {config.output_path}")

if __name__ == "__main__":
    main()

Eye tracking video saved to: assets/EyeTracking.avi
